# <center><span style="font-size: 42px;color: darkgreen;">Lab1: Modelo de Classificação</center></span>

<br>

---

<br>

## Contexto

Neste laboratório, foi fornecido o arquivo **`NaiveBayes.zip`**, contendo o script `NaiveBayes.sh` e duas pastas: `ham` e `spam`.

- A pasta `ham` contém 10 arquivos `.txt` com mensagens consideradas válidas (não spam).
- A pasta `spam` contém 10 arquivos `.txt` com mensagens consideradas spam.

O arquivo `NaiveBayes.sh` inclui os comandos necessários para a execução do laboratório.


<br>

## Problema de Negócio

> Detectar se uma mensagem é Spam ou não. 

<br><br>

---

<br><br>

# <center>Iniciando o Laboratório</center>

<br><br>





## 1 Iniciando os Serviços

<br>

- **1.1 Iniciar o HDFS (NameNode, DataNode, SecondaryNameNode)**:
   ```bash
   start-dfs.sh  |  stop-dfs.sh
   ```
- **1.2 Iniciar o YARN (ResourceManager, NodeManager)**:
   ```bash
   start-yarn.sh  |  stop-yarn.sh
   ```
- **1.3 Verificando serviços**:
   ```bash
   jps
   ```

<br><br>

## 2. Gravando os Dados no Cluster HDFS

<br>

Nesta etapa, vamos copiar os arquivos do sistema local para o `HDFS` para que possam ser processados pelo `Mahout`.

<br>

#### 2.1 Criando Diretórios no HDFS

No terminal digitar os comandos:

```bash
hdfs dfs -mkdir /mahout
hdfs dfs -mkdir /mahout/input
hdfs dfs -mkdir /mahout/input/ham
hdfs dfs -mkdir /mahout/input/spam
```

#### 2.2 Lista os arquivos e diretórios no HDFS raiz.

```bash
hdfs dfs -ls /
```

#### 2.3 Copiando dados do sistema local para o HDFS

No diretório onde estão as pastas `ham` e `spam` ir ao terminal e digitar:

```bash
hdfs dfs -copyFromLocal ham/* /mahout/input/ham
hdfs dfs -copyFromLocal spam/* /mahout/input/spam
```

#### 2.4 Verificar conteúdo

```bash
hdfs dfs -ls /mahout/input/ham
hdfs dfs -ls /mahout/input/spam
```

<br><br>


## 3. Pré-Processamento dos Dados com Apache Mahout

> **Importante**: O pré-processamento a seguir é específico para dados em formato de texto, como o conjunto de dados atual, que contém mensagens armazenadas como arquivos `.txt`. Dependendo do tipo de dados (imagens, dados numéricos, etc.), outras técnicas de pré-processamento podem ser necessárias.

> O **objetivo do pré-processamento** é converter os textos em um formato que o Mahout possa utilizar para análise e modelagem.

<br>

#### 3.1 Converter Dados para Formato de Sequence (obrigatório quando se trabalha com Mahout)

O primeiro passo no `Mahout` é transformar os dados de texto em uma **sequence file**, um formato binário que facilita o processamento distribuído no `Hadoop`.

```bash
mahout seqdirectory -i /mahout/input -o /mahout/output/seqoutput
```

<br>

#### 3.2 Criar a Matriz Esparsa com Vetores TF-IDF

Com os dados em formato de sequence, o próximo passo é converter esses arquivos para uma matriz esparsa de vetores **TF-IDF** (Term Frequency-Inverse Document Frequency). Essa matriz representa os termos em cada documento e permite que o Mahout aplique algoritmos de aprendizado de máquina nos dados.

```bash
mahout seq2sparse -i /mahout/output/seqoutput -o /mahout/output/sparseoutput
```

<br>

#### 3.2 Verificar Saída da Matriz Esparsa

Após a criação da matriz esparsa, verifique o diretório de saída para confirmar que os vetores **TF-IDF** foram gerados corretamente:

```bash
hdfs dfs -ls /mahout/output/sparseoutput
```
<br>

A saída mostra os arquivos e diretórios gerados para representar os dados de texto em formato de vetores esparsos:

- **df-count**: Contagem de documentos por termo (usado para calcular **IDF**).
- **dictionary.file-0**: Mapeamento de termos para índices únicos.
- **frequency.file-0**: Frequência de termos no conjunto de dados.
- **tf-vectors**: Vetores de frequência dos termos (**TF**) por documento.
- **tfidf-vectors**: Vetores **TF-IDF**, que indicam a relevância de cada termo.
- **tokenized-documents**: Documentos tokenizados em palavras.
- **wordcount**: Contagem total de palavras. (`hadoop fs -text /mahout/output/sparseoutput/wordcount/*`)

<br>

Esses dados indicam que o pré-processamento está completo e pronto para uso em modelos de aprendizado de máquina.

<br><br>


## 4. Divisão dos Dados em Treino e Teste

<br>

A divisão dos dados em conjuntos de treino e teste é essencial para avaliar a performance do modelo em dados novos. O Mahout permite fazer essa divisão automaticamente a partir do conjunto de vetores **TF-IDF** criado anteriormente. Nesta etapa, selecionamos 30% dos dados para o conjunto de teste, e os 70% restantes serão usados para treinar o modelo.


```bash
# Explicação de cada parâmetro
#	-i	                    pasta com dados de entrada
#	--trainingOutput	    dados de treino
#	--testOutput		    dados de teste
#	--randomSelectionPct	percentual de divisão dos dados
#	--overwrite			    overwrite
#	--sequenceFiles		    input sequencial
#	--xm				    tipo de processamento. 

mahout split -i /mahout/output/sparseoutput/tfidf-vectors --trainingOutput /mahout/nbTrain --testOutput /mahout/nbTest --randomSelectionPct 30 --overwrite --sequenceFiles -xm sequencial


# Verifique o diretório de treino
hdfs dfs -ls /mahout/nbTrain

# Verifique o diretório de teste
hdfs dfs -ls /mahout/nbTest
```

<br><br>


## 5. Treinando o Modelo com Dados no Cluster HDFS

<br>

Agora, com os dados de treino devidamente preparados, é hora de aplicar o algoritmo **Naive Bayes** usando o **Apache Mahout** ao conjunto de dados, gerando um modelo que pode classificar mensagens como **spam** ou **ham** (não spam). O modelo treinado será armazenado no `HDFS` para uso posterior nas etapas de teste e avaliação.

```bash
# Explicação de cada parâmetro
#	-i	dados de treino
#	-li onde armazenar os labels
#	-o	onde armazenar o modelo
#	-ow	overwrite
#	-c	complementary
mahout trainnb -i /mahout/nbTrain -li /mahout/nbLabels -o /mahout/nbmodel -ow -c
```

<br><br>

## 6. Testando e Avaliando o Modelo

<br>

Agora, com o modelo treinado, o próximo passo é testá-lo nos dados de teste para verificar sua precisão e eficácia. O `Mahout` utilizará o modelo salvo em `/mahout/nbmodel` para fazer previsões sobre as mensagens no conjunto de teste. O comando `testnb` gera previsões para cada entrada do conjunto de teste e salva os resultados.

```bash
# Teste do Modelo
#	-i  pasta com os dados de teste
#	-m	pasta do modelo
#	-l	labels 
#	-ow	overwrite
#	-o	pasta com as previsões
#	-c	complementary 
mahout testnb -i /mahout/nbTest -m /mahout/nbmodel -l /mahout/nbLabels -ow -o /mahout/nbpredictions -c
```
<br>

A saída contém:

- **nbpredictions**: Contém as previsões realizadas pelo modelo, indicando se cada mensagem foi classificada como spam ou ham.

<br><br><br><br>

# Fim!